## Using Python to Query MySQL
This notebook demonstrates using a couple of different database connectivity libraries to connect to and query a MySQL database.
- **PyMySQL** library
- MySQL's Native **mysql.connector** library
- **SqlAlchemy** library

### 1.0. Prerequisites

#### 1.1. First, you must **install** the libaries into your *python* environment by executing the following commands in a *Terminal window*
- \$ python3 -m pip install PyMySQL
- \$ python3 -m pip install mysql-connector-python
- \$ python3 -m pip install sqlalchemy

#### 1.2. Next, as with all Jupyter Notebooks, you need to **Import** the libaries that you'll be working with in the notebook,

In [ ]:
!pip install PyMySQL
!pip install mysql-connector-python
!pip install sqlalchemy
import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine

import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#### 1.3. And then, Assign Connection Variables for the MySQL Server & Database with which You'll be Working 

In [26]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"

user_id = "ds2002"
pwd = "UVA!1819"
db_name = "northwind"

### 2.0. Using the PyMySQL Library
#### 2.1. Using a Cursor to Iterate the Rows Returned

In [27]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

try:
    cursor.execute('SELECT * FROM products;')
    
    for row in cursor.fetchmany(size=5):
        print(row)
        
    cursor.close()
    
except:
    print ("Error: unable to fetch data")
    
conn.close()

('4', 1, 'NWTB-1', 'Northwind Traders Chai', None, Decimal('13.5000'), Decimal('18.0000'), 10, 40, '10 boxes x 20 bags', 0, 10, 'Beverages', b'')
('10', 3, 'NWTCO-3', 'Northwind Traders Syrup', None, Decimal('7.5000'), Decimal('10.0000'), 25, 100, '12 - 550 ml bottles', 0, 25, 'Condiments', b'')
('10', 4, 'NWTCO-4', 'Northwind Traders Cajun Seasoning', None, Decimal('16.5000'), Decimal('22.0000'), 10, 40, '48 - 6 oz jars', 0, 10, 'Condiments', b'')
('10', 5, 'NWTO-5', 'Northwind Traders Olive Oil', None, Decimal('16.0125'), Decimal('21.3500'), 10, 40, '36 boxes', 0, 10, 'Oil', b'')
('2;6', 6, 'NWTJP-6', 'Northwind Traders Boysenberry Spread', None, Decimal('18.7500'), Decimal('25.0000'), 25, 100, '12 - 8 oz jars', 0, 25, 'Jams, Preserves', b'')


In [28]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor(pymysql.cursors.DictCursor)

try:
    cursor.execute('SELECT * FROM products ORDER BY list_price DESC LIMIT 5;')
    
    for row in cursor.fetchall():
        print(row)
        
    cursor.close()
    
except:
    print ("Error: unable to fetch data")
    
conn.close()

{'supplier_ids': '2;6', 'id': 20, 'product_code': 'NWTJP-6', 'product_name': 'Northwind Traders Marmalade', 'description': None, 'standard_cost': Decimal('60.7500'), 'list_price': Decimal('81.0000'), 'reorder_level': 10, 'target_level': 40, 'quantity_per_unit': '30 gift boxes', 'discontinued': 0, 'minimum_reorder_quantity': 10, 'category': 'Jams, Preserves', 'attachments': b''}
{'supplier_ids': '2', 'id': 51, 'product_code': 'NWTDFN-51', 'product_name': 'Northwind Traders Dried Apples', 'description': None, 'standard_cost': Decimal('39.7500'), 'list_price': Decimal('53.0000'), 'reorder_level': 10, 'target_level': 40, 'quantity_per_unit': '50 - 300 g pkgs.', 'discontinued': 0, 'minimum_reorder_quantity': 10, 'category': 'Dried Fruit & Nuts', 'attachments': b''}
{'supplier_ids': '3;4', 'id': 43, 'product_code': 'NWTB-43', 'product_name': 'Northwind Traders Coffee', 'description': None, 'standard_cost': Decimal('34.5000'), 'list_price': Decimal('46.0000'), 'reorder_level': 25, 'target_lev

#### 2.2. Using the Pandas read_sql() Method to Return a DataFrame

In [29]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)

df = pd.read_sql("SELECT * FROM products ORDER BY list_price DESC;", conn)

conn.close()
df.head()

,supplier_ids,id,product_code,product_name,description,standard_cost,list_price,reorder_level,target_level,quantity_per_unit,discontinued,minimum_reorder_quantity,category,attachments
0,2;6,20,NWTJP-6,Northwind Traders Marmalade,None,60.75,81.0,10,40,30 gift boxes,0,10.0,"Jams, Preserves",b''
1,2,51,NWTDFN-51,Northwind Traders Dried Apples,None,39.75,53.0,10,40,50 - 300 g pkgs.,0,10.0,Dried Fruit & Nuts,b''
2,3;4,43,NWTB-43,Northwind Traders Coffee,None,34.50,46.0,25,100,16 - 500 g tins,0,25.0,Beverages,b''
3,8,8,NWTS-8,Northwind Traders Curry Sauce,None,30.00,40.0,10,40,12 - 12 oz jars,0,10.0,Sauces,b''
4,6,17,NWTCFV-17,Northwind Traders Fruit Cocktail,None,29.25,39.0,10,40,15.25 OZ,0,10.0,Canned Fruit & Vegetables,b''


In [30]:
lbound = 15.00
ubound = 20.00

sql_query = """
    SELECT id AS product_id
        , product_name
        , list_price
    FROM northwind.products
    WHERE list_price BETWEEN %s AND %s
    ORDER BY list_price DESC;
"""

In [31]:
conn = mysql.connector.connect(user=user_id, password=pwd,
                               host=host_ip, database=db_name,
                               auth_plugin='mysql_native_password') #caching_sha2_password mysql_native_password

In [32]:
cursor = conn.cursor()

try:
    cursor.execute(sql_query, (lbound, ubound))
    
    for (product_id, product_name, list_price) in cursor:
        print("{}, {} was sold for {}".format(product_id, product_name, list_price))
        
    cursor.close()
    
except:
    print ("Error: unable to fetch data")   

57, Northwind Traders Ravioli was sold for 19.5000
40, Northwind Traders Crab Meat was sold for 18.4000
1, Northwind Traders Chai was sold for 18.0000
66, Northwind Traders Tomato Sauce was sold for 17.0000
86, Northwind Traders Cake Mix was sold for 15.9900


In [33]:
cursor = conn.cursor()

try:
    cursor.execute(sql_query, (lbound, ubound))
    
    row = cursor.fetchone() 
    while row:
        print(row)
        row = cursor.fetchone()
        
    cursor.close()
    
except:
    print ("Error: unable to fetch data")

(57, 'Northwind Traders Ravioli', Decimal('19.5000'))
(40, 'Northwind Traders Crab Meat', Decimal('18.4000'))
(1, 'Northwind Traders Chai', Decimal('18.0000'))
(66, 'Northwind Traders Tomato Sauce', Decimal('17.0000'))
(86, 'Northwind Traders Cake Mix', Decimal('15.9900'))


In [34]:
conn.close()

#### 3.2. Using Pandas read_sql() Method to Return a DataFrame

In [35]:
lbound = 15.00
ubound = 20.00

sql_query = """
    SELECT id AS product_id
        , product_name
        , list_price
    FROM northwind.products
    WHERE list_price BETWEEN {0} AND {1}
    ORDER BY list_price DESC;
""".format(lbound, ubound)

print(sql_query)


    SELECT id AS product_id
        , product_name
        , list_price
    FROM northwind.products
    WHERE list_price BETWEEN 15.0 AND 20.0
    ORDER BY list_price DESC;



In [36]:
configs = {
    'user': user_id,
    'password': pwd,
    'host': host_name,
    'database': db_name,
    'auth_plugin': 'mysql_native_password',
    'raise_on_warnings': True
}

conn = mysql.connector.connect(**configs)

df = pd.read_sql(sql_query, conn)

conn.close()
df.tail()

,product_id,product_name,list_price
0,57,Northwind Traders Ravioli,19.50
1,40,Northwind Traders Crab Meat,18.40
2,1,Northwind Traders Chai,18.00
3,66,Northwind Traders Tomato Sauce,17.00
4,86,Northwind Traders Cake Mix,15.99


### 5.0 Define Helper Functions to Encapsulate and Abstract the Implementation Details

In [37]:
sql_query = """
    SELECT id AS product_id
        , product_name
        , list_price
    FROM northwind.products
    ORDER BY list_price DESC;
"""

#### 5.1. Using Individual Connection Parameters

In [38]:
def get_pymysql_dataframe(host, user, password, database_name, sql_query_string):
    connection = pymysql.connect(host=host, user=user, password=password, database=database_name)
    dframe = pd.read_sql(sql_query_string, connection)
    connection.close()
    
    return dframe

In [39]:
df = get_pymysql_dataframe(host_name, user_id, pwd, db_name, sql_query)
df.head()

,product_id,product_name,list_price
0,20,Northwind Traders Marmalade,81.0
1,51,Northwind Traders Dried Apples,53.0
2,43,Northwind Traders Coffee,46.0
3,8,Northwind Traders Curry Sauce,40.0
4,17,Northwind Traders Fruit Cocktail,39.0


In [40]:
print("Shape: {}\n".format(df.shape))

Shape: (45, 3)



#### 5.1.1. Using SqlAlchemy

In [47]:
def get_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe

In [57]:
df = get_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, sql_query)
df.head()

,supplier_ids,id,product_code,product_name,description,standard_cost,list_price,reorder_level,target_level,quantity_per_unit,discontinued,minimum_reorder_quantity,category,attachments
0,4,1,NWTB-1,Northwind Traders Chai,None,13.5000,18.00,10,40,10 boxes x 20 bags,0,10.0,Beverages,b''
1,10,3,NWTCO-3,Northwind Traders Syrup,None,7.5000,10.00,25,100,12 - 550 ml bottles,0,25.0,Condiments,b''
2,10,4,NWTCO-4,Northwind Traders Cajun Seasoning,None,16.5000,22.00,10,40,48 - 6 oz jars,0,10.0,Condiments,b''
3,10,5,NWTO-5,Northwind Traders Olive Oil,None,16.0125,21.35,10,40,36 boxes,0,10.0,Oil,b''
4,2;6,6,NWTJP-6,Northwind Traders Boysenberry Spread,None,18.7500,25.00,25,100,12 - 8 oz jars,0,25.0,"Jams, Preserves",b''


In [58]:
print(f"Shape: {df.shape[0]} Observations x {df.shape[1]} Features")

Shape: 45 Observations x 14 Features


### 6.0. Writing a Pandas DataFrame to a SQL Database

In [59]:
def insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, df, table_name):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    df.to_sql(table_name, con=connection, if_exists='replace') #, index_label='product_id');  'append'
    connection.close()

In [60]:
insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, df, 'dim_products')

In [61]:
df = get_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, 'SELECT * FROM dim_products')
df.head()

,index,supplier_ids,id,product_code,product_name,description,standard_cost,list_price,reorder_level,target_level,quantity_per_unit,discontinued,minimum_reorder_quantity,category,attachments
0,0,4,1,NWTB-1,Northwind Traders Chai,None,13.5000,18.00,10,40,10 boxes x 20 bags,0,10.0,Beverages,
1,1,10,3,NWTCO-3,Northwind Traders Syrup,None,7.5000,10.00,25,100,12 - 550 ml bottles,0,25.0,Condiments,
2,2,10,4,NWTCO-4,Northwind Traders Cajun Seasoning,None,16.5000,22.00,10,40,48 - 6 oz jars,0,10.0,Condiments,
3,3,10,5,NWTO-5,Northwind Traders Olive Oil,None,16.0125,21.35,10,40,36 boxes,0,10.0,Oil,
4,4,2;6,6,NWTJP-6,Northwind Traders Boysenberry Spread,None,18.7500,25.00,25,100,12 - 8 oz jars,0,25.0,"Jams, Preserves",
